In [ ]:
import pandas as pd 
import numpy as np
import glob
from tqdm import tqdm
import cv2
from sklearn.model_selection import train_test_split

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset
from torchvision import datasets, models, transforms
from torchvision.models import resnet50

from PIL import Image
import matplotlib.pyplot as plt
from IPython.display import clear_output

## Скачивание файлов



In [ ]:
!wget https://lodmedia.hb.bizmrg.com/case_files/868817/train_dataset_train.zip

--2022-11-07 04:34:33--  https://lodmedia.hb.bizmrg.com/case_files/868817/train_dataset_train.zip
Resolving lodmedia.hb.bizmrg.com (lodmedia.hb.bizmrg.com)... 217.69.142.100
Connecting to lodmedia.hb.bizmrg.com (lodmedia.hb.bizmrg.com)|217.69.142.100|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 19494716 (19M) [application/x-zip-compressed]
Saving to: ‘train_dataset_train.zip’

train_dataset_train 100%[===================>]  18.59M   797KB/s    in 22s     

2022-11-07 04:34:56 (875 KB/s) - ‘train_dataset_train.zip’ saved [19494716/19494716]



In [ ]:
!wget https://lodmedia.hb.bizmrg.com/case_files/868817/test_dataset_test.zip

--2022-11-07 04:34:56--  https://lodmedia.hb.bizmrg.com/case_files/868817/test_dataset_test.zip
Resolving lodmedia.hb.bizmrg.com (lodmedia.hb.bizmrg.com)... 217.69.142.100
Connecting to lodmedia.hb.bizmrg.com (lodmedia.hb.bizmrg.com)|217.69.142.100|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8387165 (8.0M) [application/x-zip-compressed]
Saving to: ‘test_dataset_test.zip’

test_dataset_test.z 100%[===================>]   8.00M  1.38MB/s    in 6.3s    

2022-11-07 04:35:03 (1.27 MB/s) - ‘test_dataset_test.zip’ saved [8387165/8387165]



In [ ]:
!unzip /content/train_dataset_train.zip

Archive:  /content/train_dataset_train.zip
  inflating: train.csv               
   creating: train/
  inflating: train/2138.jpg          
  inflating: train/2139.jpg          
  inflating: train/2140.jpg          
  inflating: train/2141.jpg          
  inflating: train/2142.jpg          
  inflating: train/2143.jpg          
  inflating: train/2144.jpg          
  inflating: train/2145.jpg          
  inflating: train/2146.jpg          
  inflating: train/2147.jpg          
  inflating: train/2148.jpg          
  inflating: train/2149.jpg          
  inflating: train/2150.jpg          
  inflating: train/2151.jpg          
  inflating: train/2152.jpg          
  inflating: train/2153.jpg          
  inflating: train/2154.jpg          
  inflating: train/2155.jpg          
  inflating: train/2156.jpg          
  inflating: train/2157.jpg          
  inflating: train/2158.jpg          
  inflating: train/2159.jpg          
  inflating: train/2160.jpg          
  inflating: train/2161.j

In [ ]:
!unzip /content/test_dataset_test.zip -d test

Archive:  /content/test_dataset_test.zip
  inflating: test/test.csv           
   creating: test/test/
  inflating: test/test/0.jpg         
  inflating: test/test/1.jpg         
  inflating: test/test/10.jpg        
  inflating: test/test/100.jpg       
  inflating: test/test/1000.jpg      
  inflating: test/test/1001.jpg      
  inflating: test/test/1002.jpg      
  inflating: test/test/1003.jpg      
  inflating: test/test/1004.jpg      
  inflating: test/test/1005.jpg      
  inflating: test/test/1006.jpg      
  inflating: test/test/1007.jpg      
  inflating: test/test/1008.jpg      
  inflating: test/test/1009.jpg      
  inflating: test/test/101.jpg       
  inflating: test/test/1010.jpg      
  inflating: test/test/1011.jpg      
  inflating: test/test/1012.jpg      
  inflating: test/test/1013.jpg      
  inflating: test/test/1014.jpg      
  inflating: test/test/1015.jpg      
  inflating: test/test/1016.jpg      
  inflating: test/test/1017.jpg      
  inflating: test/test/

## Transformers


In [ ]:
!pip install hugsvision

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.5 MB 37.8 MB/s 
     |████████████████████████████████| 548 kB 55.8 MB/s 
     |████████████████████████████████| 529 kB 73.9 MB/s 
     |████████████████████████████████| 796 kB 54.1 MB/s 
     |████████████████████████████████| 136 kB 69.3 MB/s 
     |████████████████████████████████| 87 kB 8.1 MB/s 
     |████████████████████████████████| 163 kB 58.1 MB/s 
     |████████████████████████████████| 7.6 MB 63.2 MB/s 
  Created wheel for fire: filename=fire-0.4.0-py2.py3-none-any.whl size=115942 sha256=47ecdbe4407317dde8cc04a2bb554dbe9a8f611e73456bcbc78f165a7b24d4d9
  Stored in directory: /root/.cache/pip/wheels/8a/67/fb/2e8a12fa16661b9d5af1f654bd199366799740a85c64981226
Successfully built fire


## prepare data

In [ ]:
import random
import shutil

In [ ]:
# читаем датасет
df_train = pd.read_csv("/content/train.csv")

In [ ]:
df_train['class'].value_counts()

1.0    1586
3.0     969
4.0     797
5.0     514
0.0     321
2.0     316
6.0     293
7.0     194
Name: class, dtype: int64

In [ ]:
df_train[df_train['ID_img'] == '2140.jpg']

,ID_img,class
2,2140.jpg,3.0


In [ ]:
!rm -rf train1

In [ ]:
!mkdir train1

In [ ]:
!mkdir train1/0
!mkdir train1/1
!mkdir train1/2
!mkdir train1/3
!mkdir train1/4
!mkdir train1/5
!mkdir train1/6
!mkdir train1/7

In [ ]:
for index, row in df_train.iterrows():
    folder_name = str(int(row['class']))
    
    shutil.copyfile('/content/train/' + row['ID_img'], '/content/train1/' + folder_name + '/' + row['ID_img'])


In [ ]:
!mv /content/train1/4/3819.jpg /content/train1/5/3819.jpg

In [ ]:
!mv /content/train1/2/4281.jpg /content/train1/1/4281.jpg

In [ ]:
!mv /content/train1/5/3166.jpg /content/train1/4/3166.jpg

In [ ]:
!pip install -U albumentations

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 123 kB 26.5 MB/s 
  Attempting uninstall: albumentations
    Found existing installation: albumentations 1.2.1
    Uninstalling albumentations-1.2.1:
      Successfully uninstalled albumentations-1.2.1


In [ ]:
import os
import albumentations as A

In [ ]:
transform = A.Compose([
      A.CLAHE(),
      A.Flip(),
      # A.Transpose(),
      # A.ShiftScaleRotate(shift_limit=0.0625, scale_limit=0.50, rotate_limit=0, p=.75),
      A.Blur(blur_limit=3),
      A.OpticalDistortion(),
      A.GridDistortion(),
      A.HueSaturationValue(),
  ])
random.seed(42)
def augmentate(dir_path):
  for img in os.listdir(dir_path):
    image = cv2.imread(dir_path + img)

    augmented_image = transform(image=image)['image']
    cv2.imwrite(dir_path + img.replace('.jpg', '_aug.jpg'), augmented_image)

In [ ]:
augmentate('/content/train1/0/')

In [ ]:
augmentate('/content/train1/2/')
augmentate('/content/train1/5/')
augmentate('/content/train1/6/')
augmentate('/content/train1/7/')

In [ ]:
from hugsvision.dataio.VisionDataset import VisionDataset

train, test, id2label, label2id = VisionDataset.fromImageFolder(
  "/content/train1/",
  test_ratio   = 0.0,
  balanced     = True,
  augmentation = True,
)

/usr/local/lib/python3.7/dist-packages/torchvision/transforms/transforms.py:333: UserWarning: Argument 'interpolation' of type int is deprecated since 0.13 and will be removed in 0.15. Please use InterpolationMode enum.
  "Argument 'interpolation' of type int is deprecated since 0.13 and will be removed in 0.15. "
/usr/local/lib/python3.7/dist-packages/torchvision/transforms/transforms.py:333: UserWarning: Argument 'interpolation' of type int is deprecated since 0.13 and will be removed in 0.15. Please use InterpolationMode enum.
  "Argument 'interpolation' of type int is deprecated since 0.13 and will be removed in 0.15. "


Split Datasets...
Balance train dataset...
The less represented label in train as 388 occurrences
Size of train after balancing is 3104
train_ds:  3104
+---------+-----+-----+-----+-----+-----+-----+-----+-----+-------+
| Dataset |  0  |  1  |  2  |  3  |  4  |  5  |  6  |  7  | Total |
+---------+-----+-----+-----+-----+-----+-----+-----+-----+-------+
|  Train  | 388 | 388 | 388 | 388 | 388 | 388 | 388 | 388 | 3104  |
|  Test   |  0  |  0  |  0  |  0  |  0  |  0  |  0  |  0  |   0   |
+---------+-----+-----+-----+-----+-----+-----+-----+-----+-------+


In [ ]:
from hugsvision.nnet.VisionClassifierTrainer import VisionClassifierTrainer
from transformers import ViTFeatureExtractor, ViTForImageClassification, DeiTFeatureExtractor, DeiTForImageClassification, BeitFeatureExtractor, BeitForImageClassification

In [ ]:
huggingface_model = "facebook/deit-base-distilled-patch16-224"
# huggingface_model = "facebook/deit-base-distilled-patch16-384"
# huggingface_model = 'google/vit-base-patch16-224'
# huggingface_model = 'microsoft/beit-base-patch16-224-pt22k-ft22k'

# Train the model
trainer = VisionClassifierTrainer(
	model_name   = 'deit_model',
	train      	 = train,
	test      	 = test,
	output_dir   = '/content/out/',

	max_epochs   = 30,
	batch_size   = 32, 
    lr 		     = 0.0001,
	fp16	     = True,
	model = DeiTForImageClassification.from_pretrained(
	    huggingface_model,
	    num_labels = len(label2id),
	    label2id   = label2id,
	    id2label   = id2label,
			# ignore_mismatched_sizes=True
	),
	feature_extractor = DeiTFeatureExtractor.from_pretrained(
		huggingface_model,
	),
)

Downloading:   0%|          | 0.00/69.6k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/349M [00:00<?, ?B/s]

Some weights of the model checkpoint at facebook/deit-base-distilled-patch16-224 were not used when initializing DeiTForImageClassification: ['cls_classifier.bias', 'distillation_classifier.weight', 'distillation_classifier.bias', 'cls_classifier.weight']
- This IS expected if you are initializing DeiTForImageClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DeiTForImageClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DeiTForImageClassification were not initialized from the model checkpoint at facebook/deit-base-distilled-patch16-224 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream t

Downloading:   0%|          | 0.00/287 [00:00<?, ?B/s]

{'0': '0', '1': '1', '2': '2', '3': '3', '4': '4', '5': '5', '6': '6', '7': '7'}
{'0': '0', '1': '1', '2': '2', '3': '3', '4': '4', '5': '5', '6': '6', '7': '7'}


Using cuda_amp half precision backend
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 3104
  Num Epochs = 30
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 2910
  Number of trainable parameters = 85806344


Trainer builded!
Start Training!


Epoch,Training Loss,Validation Loss
1,No log,No log
2,No log,No log
3,No log,No log
4,No log,No log
5,No log,No log
6,0.067400,No log
7,0.067400,No log
8,0.067400,No log
9,0.067400,No log
10,0.067400,No log


***** Running Evaluation *****
  Num examples = 0
  Batch size = 32
***** Running Evaluation *****
  Num examples = 0
  Batch size = 32
***** Running Evaluation *****
  Num examples = 0
  Batch size = 32
***** Running Evaluation *****
  Num examples = 0
  Batch size = 32
***** Running Evaluation *****
  Num examples = 0
  Batch size = 32
***** Running Evaluation *****
  Num examples = 0
  Batch size = 32
***** Running Evaluation *****
  Num examples = 0
  Batch size = 32
***** Running Evaluation *****
  Num examples = 0
  Batch size = 32
***** Running Evaluation *****
  Num examples = 0
  Batch size = 32
***** Running Evaluation *****
  Num examples = 0
  Batch size = 32
***** Running Evaluation *****
  Num examples = 0
  Batch size = 32
***** Running Evaluation *****
  Num examples = 0
  Batch size = 32
***** Running Evaluation *****
  Num examples = 0
  Batch size = 32
***** Running Evaluation *****
  Num examples = 0
  Batch size = 32
***** Running Evaluation *****
  Num examples = 

Model saved at: /content/out/DEIT_MODEL/30_2022-11-07-04-46-37


In [ ]:
ref, hyp = trainer.evaluate_f1_score()

0it [00:00, ?it/s]

              precision    recall  f1-score   support

           0     0.0000    0.0000    0.0000       0.0
           1     0.0000    0.0000    0.0000       0.0
           2     0.0000    0.0000    0.0000       0.0
           3     0.0000    0.0000    0.0000       0.0
           4     0.0000    0.0000    0.0000       0.0
           5     0.0000    0.0000    0.0000       0.0
           6     0.0000    0.0000    0.0000       0.0
           7     0.0000    0.0000    0.0000       0.0

   micro avg     0.0000    0.0000    0.0000       0.0
   macro avg     0.0000    0.0000    0.0000       0.0
weighted avg     0.0000    0.0000    0.0000       0.0

Logs saved at: /content/out/DEIT_MODEL/30_2022-11-07-04-46-37


In [ ]:
!zip -r /content/model_30.zip /content/out/DEIT_MODEL/30_2022-11-07-04-46-37

  adding: content/out/DEIT_MODEL/30_2022-11-07-04-46-37/ (stored 0%)
  adding: content/out/DEIT_MODEL/30_2022-11-07-04-46-37/1667796403.2451177/ (stored 0%)
  adding: content/out/DEIT_MODEL/30_2022-11-07-04-46-37/1667796403.2451177/events.out.tfevents.1667796403.61a588b5de7b.75.1 (deflated 62%)
  adding: content/out/DEIT_MODEL/30_2022-11-07-04-46-37/feature_extractor/ (stored 0%)
  adding: content/out/DEIT_MODEL/30_2022-11-07-04-46-37/feature_extractor/preprocessor_config.json (deflated 44%)
  adding: content/out/DEIT_MODEL/30_2022-11-07-04-46-37/logs.txt (deflated 83%)
  adding: content/out/DEIT_MODEL/30_2022-11-07-04-46-37/trainer/ (stored 0%)
  adding: content/out/DEIT_MODEL/30_2022-11-07-04-46-37/trainer/pytorch_model.bin (deflated 7%)
  adding: content/out/DEIT_MODEL/30_2022-11-07-04-46-37/trainer/training_args.bin (deflated 47%)
  adding: content/out/DEIT_MODEL/30_2022-11-07-04-46-37/trainer/config.json (deflated 55%)
  adding: content/out/DEIT_MODEL/30_2022-11-07-04-46-37/events

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!cp /content/model_30.zip /content/drive/MyDrive/AI/hacks-ai/chemp1/model_30.zip

Модель сохраняется в отдельной папке, поэтому нужно поменять путь, можно взять из предыдущей ячейки новый путь

In [ ]:
import os.path
from transformers import ViTFeatureExtractor, ViTForImageClassification, DeiTFeatureExtractor, DeiTForImageClassification, BeitFeatureExtractor, BeitForImageClassification
from hugsvision.inference.VisionClassifierInference import VisionClassifierInference

path = '/content/out/DEIT_MODEL/30_2022-11-07-04-46-37/'
model_path = path + 'model'
extractor_path = path + 'feature_extractor'

classifier = VisionClassifierInference(
    feature_extractor = DeiTFeatureExtractor.from_pretrained(extractor_path),
    model = DeiTForImageClassification.from_pretrained(model_path),
    # resolution = 384
)


# classifier = VisionClassifierInference(
#     feature_extractor = BeitFeatureExtractor.from_pretrained(extractor_path),
#     model = BeitForImageClassification.from_pretrained(path),
#     # resolution = 384
# )

loading configuration file /content/out/DEIT_MODEL/30_2022-11-07-04-46-37/feature_extractor/preprocessor_config.json
Feature extractor DeiTFeatureExtractor {
  "crop_size": 224,
  "do_center_crop": true,
  "do_normalize": true,
  "do_resize": true,
  "feature_extractor_type": "DeiTFeatureExtractor",
  "image_mean": [
    0.485,
    0.456,
    0.406
  ],
  "image_std": [
    0.229,
    0.224,
    0.225
  ],
  "resample": 3,
  "size": 256
}

loading configuration file /content/out/DEIT_MODEL/30_2022-11-07-04-46-37/model/config.json
Model config DeiTConfig {
  "_name_or_path": "facebook/deit-base-distilled-patch16-224",
  "architectures": [
    "DeiTForImageClassification"
  ],
  "attention_probs_dropout_prob": 0.0,
  "encoder_stride": 16,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.0,
  "hidden_size": 768,
  "id2label": {
    "0": "0",
    "1": "1",
    "2": "2",
    "3": "3",
    "4": "4",
    "5": "5",
    "6": "6",
    "7": "7"
  },
  "image_size": 224,
  "initializer_range": 0

Model loaded!


In [60]:
image = Image.open('/content/test/test/1.jpg')
# image normlize with 224 x 224
# new_image = image.resize((384, 384))

In [61]:
label = classifier.predict_image(image)
print("Predicted class:", label)

Predicted class: 5


In [62]:
label = classifier.predict(img_path='/content/test/test/0.jpg')
print("Predicted class:", label)

Predicted class: 5


In [63]:
test_df = pd.read_csv("/content/test/test.csv")
# test_df = test_df.drop(["class"], axis = 1)

In [64]:
test_df.head(10)

,ID_img,class
0,0.jpg,0
1,1.jpg,0
2,2.jpg,0
3,3.jpg,0
4,4.jpg,0
5,5.jpg,0
6,6.jpg,0
7,7.jpg,0
8,8.jpg,0
9,9.jpg,0


#Предсказываем нашей моделью результат

In [65]:
predicts = []

for idx, row in tqdm(test_df.iterrows()):
  img_path = '/content/test/test/' + row['ID_img'];
  label = classifier.predict(img_path=img_path)
  predicts.append(label)

2138it [21:27,  1.66it/s]


In [66]:
len(predicts)

2138

In [67]:
test_df['class'] = predicts


In [68]:
test_df

,ID_img,class
0,0.jpg,5
1,1.jpg,5
2,2.jpg,1
3,3.jpg,1
4,4.jpg,6
...,...,...
2133,2133.jpg,4
2134,2134.jpg,1
2135,2135.jpg,1
2136,2136.jpg,3


In [69]:
test_df.to_csv("submit20.csv", index=False)